<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/SeriesdeTiempo/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
from google.colab import userdata

# Definir user key para la API
user_key = userdata.get('ALPHACAST')

In [26]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [27]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - current_prices_mom']

In [50]:
y

,Emae - current_prices_mom
Date,
2016-02-01,-0.381524
2016-03-01,11.812005
2016-04-01,2.107285
2016-05-01,6.696828
2016-06-01,-6.024889
...,...
2024-03-01,6.438440
2024-04-01,2.222596
2024-05-01,6.341266


In [29]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')


ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)

In [30]:
ipi.head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Azúcar, productos de confitería y chocolate",50.680600,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne aviar,212.009238,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne vacuna,83.169668,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Fiambres y embutidos,143.350871,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Galletitas, productos de panadería y pastas",95.801964,NaN,NaN,NaN


In [34]:
ipi[(ipi['Seasonal Adjustment']== "Seasonally Adjusted")].head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,132.565049,NaN,NaN,NaN
2016-02-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,135.260212,NaN,NaN,2.033088
2016-03-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,132.734580,NaN,NaN,-1.867240
2016-04-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,129.439709,NaN,NaN,-2.482300
2016-05-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,128.673810,NaN,NaN,-0.591704


In [35]:
ipi = ipi[(ipi['Division'] == "IPI Manufacturero") & (ipi['Seasonal Adjustment'] == "Seasonally Adjusted")]
ipi = ipi['Value - current_prices_mom']
ipi.columns = ['Value - sa_MoM']
ipi = ipi.loc['2016-02-01':]

In [36]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')


isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_MoM']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [37]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [38]:
X = pd.concat([ipi, isac], axis=1)

In [39]:
X

,IPI,ISAC
Date,,
2016-02-01,2.033088,1.318163
2016-03-01,-1.867240,-4.613725
2016-04-01,-2.482300,-5.956458
2016-05-01,-0.591704,2.170675
2016-06-01,-1.918772,-3.266693
...,...,...
2024-03-01,-3.126307,-21.967708
2024-04-01,0.069197,10.114281
2024-05-01,-0.132622,3.534163


In [40]:
y.shape

(102,)

In [41]:
X.shape

(102, 2)

In [44]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y[:-1]) #Si ya salió IPI e ISAC, pero todavía no el EMAE entonces hay que fitear lr.fit(X[:-1], y) si ya salió todo, lr.fit(X[:-1], y[:-1])
Yest = lr.predict(X[:-1])

In [45]:
Yest

array([ 8.24014003e-01, -6.65081630e-01, -9.33502185e-01,  2.21517133e-01,
       -5.67175764e-01, -1.05058741e-01,  7.38531853e-01,  9.92451127e-01,
       -8.33898788e-01,  1.67454937e+00,  3.03956567e-01,  1.87691709e-01,
       -1.29127037e+00,  2.09739642e+00,  5.10909909e-02,  7.47918924e-01,
        1.29394363e+00,  1.84903064e-01,  2.96792946e-01,  3.07196045e-01,
        2.55803911e-01,  1.41486270e+00, -7.29376882e-01, -3.07282907e-01,
       -8.59335387e-01,  1.85320849e+00, -3.27811705e-01, -5.17120598e-01,
       -9.30178217e-01, -2.85875710e-01,  5.14691151e-01, -7.72603526e-01,
       -3.01835724e-02, -8.37054632e-01, -2.21320772e-01,  1.00780345e+00,
        7.96373358e-01, -8.25307846e-01,  1.56680841e+00,  4.02473386e-02,
       -7.38736090e-01,  7.71117938e-01, -2.57887611e-01, -5.86810086e-01,
        7.21898028e-01, -9.21469456e-01,  3.17386539e-01,  2.31395474e-01,
        3.57979425e-01, -8.02718695e+00, -7.73297868e+00,  1.17841550e+01,
        9.10316351e+00,  

In [18]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y[:-1]))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[0.03973653 0.14462032]
Valor de la intersección o coeficiente "b":
-4.800731654586717

La ecuación del modelo es igual a:
y =  [0.03973653 0.14462032] X +  -4.800731654586717

Precisión del modelo:
0.14650639187712133


In [19]:
lr.coef_[0]

0.03973652644450918

In [51]:
X[-2:-1]

,IPI,ISAC
Date,,
2024-06-01,-1.39919,-3.383952


In [48]:
emae_predict=X[-2:-1]['IPI']*lr.coef_[0]+X[-2:-1]['ISAC']*lr.coef_[1]+lr.intercept_

In [49]:
emae_predict

,0
Date,
2024-06-01,-0.443663
